In [ ]:
from api_tools.pop.tk import *

# For mugsy alignment

## convert maf to tab

In [ ]:

maf = ''
name2length = {}

block_dict = parse_maf(maf,name2length)
write_block(block_dict, 
            otab, 
            otab.replace('.tab', '_block_aln'),
            min_length=500,
            min_genomes=37)



## Convert SNP along with the concatenated alignment to position at reference genome

In [ ]:
# ! get SNP along the concatenated alignment
MC2SNP_pos = {}
MC2total_length = {}
for aln in tqdm(glob('./MC*_LCB/*_concat.aln')):
    mc = aln.split('/')[-2].split('_LCB')[0]
    records = list(SeqIO.parse(aln, 'fasta'))
    length = len(records[0].seq)
    MC2total_length[mc] = length
    vcf = aln.replace('_concat.aln', '_snp.vcf')
    df = pd.read_csv(vcf, sep='\t', comment='#', header=None)
    MC2SNP_pos[mc] = list(df[1])
    # output a new concat.aln based on the given reference. and get a new 'snp.vcf'
    # this snp.vcf is based on a concat.aln that match the reference. (in row of reference , no gap)

# ! map the concatenated aln back to its original genome coordinates
for mc in tqdm(MC2SNP_pos):
    block_tab = f'./{mc}.tab'
    block_order = f"./{mc}_LCB/block_order.list"
    if mc not in mc2ref:continue
    ref = mc2ref[mc]
    aln = f'./{mc}_LCB/{mc}_concat.aln'
    pos_list = MC2SNP_pos[mc]
    pos2pos = list(aln_coordinate_convertor(aln, ref, pos_list))
    ofile = f"./{mc}_LCB/{mc}_snp_coordinate_{ref}.tsv"
    #if not exists(ofile):
    map_back_pos(pos2pos, block_order, block_tab, ref, genome2contig2size,ofile=ofile )

for map_file in tqdm(glob(f"./*_LCB/*_snp_coordinate_*.tsv")):
    mc = map_file.split('/')[-2].split('_LCB')[0]
    block_tab = f'./{mc}.tab'
    ofile = map_file.replace('coordinate', 'density')
    summary_SNP_pos(map_file, block_tab, ofile)
